In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging

# ---------------------------------------------------------
# 1. 解决路径问题：将当前目录及子目录加入 Python 搜索路径
# ---------------------------------------------------------
# 获取当前 notebook 所在的绝对路径
current_dir = os.getcwd()

# 将根目录及其子模块目录加入 sys.path
# 这样你就可以直接 import config, PPO, factor 等
sys.path.append(current_dir)
sys.path.append(os.path.join(current_dir, 'config'))
sys.path.append(os.path.join(current_dir, 'factor'))
sys.path.append(os.path.join(current_dir, 'model'))
sys.path.append(os.path.join(current_dir, 'utils'))
sys.path.append(os.path.join(current_dir, 'backtest'))
sys.path.append(os.path.join(current_dir, 'PPO'))

# ---------------------------------------------------------
# 2. 导入你的项目模块
# ---------------------------------------------------------
from config import TrainingConfig
from miner_new import OptimizedSynergisticFactorMiner
from backtest import MinerBacktester
from utils import setup_logging

# 设置日志显示
logger = setup_logging(level=logging.INFO)

print("✅ 环境设置完成，模块导入成功！")

✅ 环境设置完成，模块导入成功！


In [2]:
# ---------------------------------------------------------
# 准备数据：使用改进的数据生成器
# 如果你有真实数据，请替换为 pd.read_csv('your_data.csv')
# ---------------------------------------------------------

from data_generator import generate_realistic_market_data, validate_data

# 1. 加载数据
print("⏳ 正在生成/加载数据...")

# 🔥 使用改进的数据生成器 - 避免未来函数，更贴近真实市场
data = generate_realistic_market_data(
    n_rows=50000,
    start_date='2023-01-01',
    freq='15min',
    seed=42,
    initial_price=10000.0
)

# 如果使用真实数据，取消下面这行的注释：
# data = pd.read_csv('your_data.csv', parse_dates=['timestamp'], index_col='timestamp')

# 2. 验证数据质量
validation = validate_data(data)
if validation['valid']:
    print("✅ 数据验证通过")
else:
    print("❌ 数据验证失败:")
    for issue in validation['issues']:
        print(f"  - {issue}")

print(f"✅ 数据加载完成，共 {len(data)} 条 K 线")
print("\n数据预览:")
print(data.tail())
print("\n数据统计:")
print(data.describe())

⏳ 正在生成/加载数据...
✅ 数据验证通过
✅ 数据加载完成，共 50000 条 K 线

数据预览:
                            open         high          low        close  \
2024-06-04 18:45:00  8027.740253  8027.795383  8021.697114  8024.287790   
2024-06-04 19:00:00  8020.396092  8037.271120  8013.764292  8019.504172   
2024-06-04 19:15:00  8019.618024  8022.761720  8018.977301  8022.585450   
2024-06-04 19:30:00  8024.479648  8027.465507  8020.015944  8020.702462   
2024-06-04 19:45:00  8020.969934  8023.650405  8016.480666  8021.491986   

                          volume  
2024-06-04 18:45:00  5556.001022  
2024-06-04 19:00:00  8782.726206  
2024-06-04 19:15:00  5189.070753  
2024-06-04 19:30:00  5095.042530  
2024-06-04 19:45:00  9896.427236  

数据统计:
               open          high           low         close        volume
count  5.000000e+04  5.000000e+04  5.000000e+04  5.000000e+04  50000.000000
mean   1.740843e+06  1.743351e+06  1.738321e+06  1.740840e+06   7646.531166
std    2.488327e+06  2.491426e+06  2.485206e+06  2

In [3]:
# ---------------------------------------------------------
# 配置训练参数
# ---------------------------------------------------------
config = TrainingConfig()

# --- 根据 Notebook 运行环境进行微调 ---
config.ppo_epochs = 4           # PPO 每次更新的轮数
config.batch_size = 64          # 批次大小
config.lr_actor = 3e-4         # 学习率
config.combiner_type = 'linear' # 'linear' (线性组合) 或 'lstm' (神经网络组合)

# 挖掘参数
BATCH_SIZE = 16                 # 每次生成的公式数量
TRAIN_INTERVAL = 100             # 每多少步训练一次 PPO

print(f"⚙️ 配置已加载: {config.combiner_type} 模式")

⚙️ 配置已加载: linear 模式


In [4]:
# ---------------------------------------------------------
# 初始化因子挖掘器
# ---------------------------------------------------------
miner = OptimizedSynergisticFactorMiner(
    data=data,
    target_col='future_return', # 目标列名
    config=config,
    max_factors=100,             # 因子池最大保留数量
    max_expr_len=50             # 因子公式最大长度
)
print("🤖 挖掘器初始化完成，准备开始挖掘...")

2025-11-21 14:22:49,730 - INFO - Calculating target variable: future_return
2025-11-21 14:22:49,733 - INFO - Target calculated: 10-period forward percentage returns
2025-11-21 14:22:49,733 - INFO -   Mean: 0.000062
2025-11-21 14:22:49,734 - INFO -   Std: 0.014640
2025-11-21 14:22:49,738 - INFO - Data split with Purging Gap=10:
2025-11-21 14:22:49,738 - INFO -   Train: 30000 bars
2025-11-21 14:22:49,739 - INFO -   Val: 10000 bars
2025-11-21 14:22:49,739 - INFO -   Test: 9980 bars
2025-11-21 14:22:50,441 - INFO - Synergy Evaluator initialized (Target: Incremental Sharpe)
2025-11-21 14:22:50,441 - INFO - Advanced Reward Calculator initialized:
2025-11-21 14:22:50,441 - INFO -   - Incremental Sharpe: False
2025-11-21 14:22:50,441 - INFO -   - Penalty: True
2025-11-21 14:22:50,442 - INFO -   - Rolling Stability: False
2025-11-21 14:22:50,442 - INFO - ✅ AdvancedRewardCalculator enabled (penalty mode)
2025-11-21 14:22:50,442 - INFO - FactorMinerCore initialized:
2025-11-21 14:22:50,442 - INFO

🤖 挖掘器初始化完成，准备开始挖掘...


In [5]:
# ---------------------------------------------------------
# 开始运行挖掘循环
# ---------------------------------------------------------
import time
start_time = time.time()

results = miner.mine_factors(
    n_iterations=1000,
    batch_size=16,
    train_interval=100,
    print_interval=5,           # 每5次打印一次进度
    early_stop_patience=1000      # 早停耐心值
)

end_time = time.time()
print(f"\n🎉 挖掘完成！耗时: {end_time - start_time:.2f} 秒")

2025-11-21 14:22:50,447 - INFO - Starting factor mining:
2025-11-21 14:22:50,448 - INFO -   Iterations: 1000
2025-11-21 14:22:50,448 - INFO -   Batch size: 16
2025-11-21 14:22:50,448 - INFO -   Early stop patience: 1000
2025-11-21 14:22:51,927 - INFO - ❌ Batch iteration 0: NO valid candidates
2025-11-21 14:22:51,928 - INFO -    All 16 expressions failed validation
2025-11-21 14:22:51,928 - INFO -    Failure breakdown:
2025-11-21 14:22:51,928 - INFO -      train_computation_failed: 16/16
2025-11-21 14:22:51,929 - INFO - 🔄 PPO Update #1 at iteration 0
2025-11-21 14:22:51,929 - INFO -   Buffer size: 736
2025-11-21 14:22:54,246 - INFO -   PPO Training Stats:
2025-11-21 14:22:54,247 - INFO -     Policy Loss: 0.107600
2025-11-21 14:22:54,247 - INFO -     Value Loss: 0.298380
2025-11-21 14:22:54,247 - INFO -     Entropy Loss: -0.068988
2025-11-21 14:22:54,247 - INFO -     Advantage - Mean: 0.6817, Std: 0.6892
2025-11-21 14:22:54,248 - INFO -     Value - Mean: -1.5887, Std: 0.6287
2025-11-21 1

KeyboardInterrupt: 

In [ ]:
# ---------------------------------------------------------
# 绘制训练历史曲线
# ---------------------------------------------------------
%matplotlib inline

print("📊 正在绘制训练历史...")
miner.plot_training_history()

# 分析性能衰退情况
miner.analyze_performance_degradation()

In [ ]:
# ---------------------------------------------------------
# 对最佳因子组合进行回测
# ---------------------------------------------------------

# 创建回测器
backtester = MinerBacktester(miner)

# 在测试集上运行回测并画图
print("\n📈 开始测试集回测...")
test_results = backtester.run(
    data_split='test', 
    top_n=5, 
    save_path='test_backtest_result.png'
)

# 打印详细的单因子表现
print("\n📋 最佳单因子表现 (Top 5):")
individual_res = backtester.get_individual_results('test')
if individual_res:
    sorted_factors = sorted(individual_res.items(), key=lambda x: x[1]['sharpe_ratio'], reverse=True)[:5]
    for name, metrics in sorted_factors:
        print(f"  {name}: Sharpe={metrics['sharpe_ratio']:.2f}, IC={metrics['ic']:.4f}")